# AIC 2024/2025 Official – Inference + Evaluation
This Colab runs host inference, writes per-query CSVs named `{query_id}.csv`, and evaluates with the official scoring.

In [ ]:
# System check
!nvidia-smi || true
!python --version
import os, sys, pathlib; print('CWD:', os.getcwd())

In [ ]:
# Setup: Clone repo and install deps (auto FAISS CPU/GPU)
REPO_URL = 'https://github.com/nqvu-daniel/AIC_FTML_dev.git'
REPO_NAME = 'AIC_FTML_dev'
import subprocess, pathlib, sys, os
if not pathlib.Path(REPO_NAME).exists():
    subprocess.run(['git', 'clone', REPO_URL], check=True)
%cd {REPO_NAME}
!python -m pip install -q -r requirements.txt

# Install FAISS based on CUDA availability
try:
    import torch
    if torch.cuda.is_available():
        print("GPU detected, installing faiss-gpu-cu12...")
        !pip install -q faiss-gpu-cu12
        print('Installed faiss-gpu-cu12 (CUDA 12 compatible)')
    else:
        print("No GPU detected, installing faiss-cpu...")
        !pip install -q faiss-cpu
        print('Installed faiss-cpu')
except Exception as e:
    print(f'FAISS install error: {e}')
    !pip install -q faiss-cpu
    print('Fallback: Installed faiss-cpu')
    
print('Setup done')

## Inference (KIS/VQA) – Official File Naming
- Provide `ARTIFACTS_BUNDLE_URL` and/or `RERANKER_MODEL_URL` if needed.
- Output path auto-uses `submissions/{query_id}.csv`.

In [ ]:
# Configure your query
QUERY = 'a person opening a laptop'
QUERY_ID = 'q1'  # official id
TASK = 'kis'     # 'kis' or 'vqa'
ANSWER = ''      # required if TASK='vqa'
ARTIFACTS_BUNDLE_URL = ''  # optional
RERANKER_MODEL_URL = ''    # optional

# Run inference
cmd = ['python', 'src/retrieval/use.py', '--query', QUERY, '--query_id', QUERY_ID, '--task', TASK]
if TASK == 'vqa':
    assert ANSWER, 'For TASK=vqa, set ANSWER'
    cmd += ['--answer', ANSWER]
if ARTIFACTS_BUNDLE_URL:
    cmd += ['--bundle_url', ARTIFACTS_BUNDLE_URL]
if RERANKER_MODEL_URL:
    cmd += ['--model_url', RERANKER_MODEL_URL]
print('Running:', ' '.join(cmd))
import subprocess
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
if res.returncode:
    print(res.stderr)
    raise SystemExit(res.returncode)
!ls -l submissions | head -n 5

## Evaluate
Place/point to your ground truth JSON and select task.

In [ ]:
# Path to ground truth (e.g., from Drive)
GT_PATH = 'ground_truth.json'  # update to your path
TASK_EVAL = TASK               # 'kis' or 'vqa' or 'trake'
NORMALIZE_ANS = False          # set True if needed for VQA

cmd = ['python', 'eval/evaluate.py', '--gt', GT_PATH, '--pred_dir', 'submissions', '--task', TASK_EVAL]
if NORMALIZE_ANS:
    cmd.append('--normalize_answer')
print('Evaluating:', ' '.join(cmd))
import subprocess
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
if res.returncode:
    print(res.stderr)
    raise SystemExit(res.returncode)
